# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,7.81,66,20,4.63,AR,1699597010
1,1,ribeira grande,38.5167,-28.7000,20.99,96,58,9.39,PT,1699597284
2,2,albany,42.6001,-73.9662,6.56,82,100,2.46,US,1699597285
3,3,thompson,55.7435,-97.8558,-10.91,92,100,0.00,CA,1699597286
4,4,grytviken,-54.2811,-36.5092,1.87,95,100,2.79,GS,1699597287


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City",
    size = "Humidity"
  
)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 27
wind_speed_s = city_data_df["Wind Speed"] > 5

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]

# # Drop any rows with null values
clean_city_temp_range_df = city_temp_range_df.dropna()

# # Display sample data
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,adamstown,-25.0660,-130.1015,22.49,83,100,6.43,PN,1699597303
51,51,al muwayh,22.4333,41.7583,22.91,29,85,7.67,SA,1699597308
60,60,wailua homesteads,22.0669,-159.3780,25.44,81,40,9.26,US,1699597313
64,64,uturoa,-16.7333,-151.4333,26.37,85,52,5.95,PF,1699597314
80,80,cabedelo,-6.9811,-34.8339,24.37,76,14,6.08,BR,1699597321
92,92,lihue,21.9789,-159.3672,25.77,76,40,9.26,US,1699597327
190,190,port mathurin,-19.6833,63.4167,24.98,75,30,6.78,MU,1699597370
213,213,puerto baquerizo moreno,-0.9000,-89.6000,24.35,83,84,6.97,EC,1699597376
244,244,rawatsar,29.2833,74.3833,25.84,38,61,5.14,IN,1699597387
314,314,forster,-32.1667,152.5167,22.43,77,13,7.46,AU,1699597418


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,42,adamstown,-25.0660,-130.1015,22.49,83,100,6.43,PN,1699597303,
51,51,al muwayh,22.4333,41.7583,22.91,29,85,7.67,SA,1699597308,
60,60,wailua homesteads,22.0669,-159.3780,25.44,81,40,9.26,US,1699597313,
64,64,uturoa,-16.7333,-151.4333,26.37,85,52,5.95,PF,1699597314,
80,80,cabedelo,-6.9811,-34.8339,24.37,76,14,6.08,BR,1699597321,
92,92,lihue,21.9789,-159.3672,25.77,76,40,9.26,US,1699597327,
190,190,port mathurin,-19.6833,63.4167,24.98,75,30,6.78,MU,1699597370,
213,213,puerto baquerizo moreno,-0.9000,-89.6000,24.35,83,84,6.97,EC,1699597376,
244,244,rawatsar,29.2833,74.3833,25.84,38,61,5.14,IN,1699597387,
314,314,forster,-32.1667,152.5167,22.43,77,13,7.46,AU,1699597418,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
al muwayh - nearest hotel: فندق ومطعم الموية
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
uturoa - nearest hotel: Hawaiki Nui hotel
cabedelo - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
port mathurin - nearest hotel: Escale Vacances
puerto baquerizo moreno - nearest hotel: Dory's House
rawatsar - nearest hotel: r h rwt
forster - nearest hotel: No hotel found
geneina - nearest hotel: Princess
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
geraldton - nearest hotel: Ocean Centre Hotel
lautoka - nearest hotel: Cathay Hotel
ak'ordat - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
nuku'alofa - nearest hotel: City Hotel
dayrut - nearest hotel: No hotel found
semera - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,42,adamstown,-25.0660,-130.1015,22.49,83,100,6.43,PN,1699597303,No hotel found
51,51,al muwayh,22.4333,41.7583,22.91,29,85,7.67,SA,1699597308,فندق ومطعم الموية
60,60,wailua homesteads,22.0669,-159.3780,25.44,81,40,9.26,US,1699597313,Hilton Garden Inn Kauai Wailua Bay
64,64,uturoa,-16.7333,-151.4333,26.37,85,52,5.95,PF,1699597314,Hawaiki Nui hotel
80,80,cabedelo,-6.9811,-34.8339,24.37,76,14,6.08,BR,1699597321,No hotel found
92,92,lihue,21.9789,-159.3672,25.77,76,40,9.26,US,1699597327,Kauai Palms
190,190,port mathurin,-19.6833,63.4167,24.98,75,30,6.78,MU,1699597370,Escale Vacances
213,213,puerto baquerizo moreno,-0.9000,-89.6000,24.35,83,84,6.97,EC,1699597376,Dory's House
244,244,rawatsar,29.2833,74.3833,25.84,38,61,5.14,IN,1699597387,r h rwt
314,314,forster,-32.1667,152.5167,22.43,77,13,7.46,AU,1699597418,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name","Country"]   
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)